In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
import random
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC
from imblearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer, RobustScaler
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split, cross_validate
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from yellowbrick.classifier.rocauc import roc_auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from yellowbrick.classifier import ROCAUC
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import roc_curve, auc
random.seed(2814)

In [2]:
df = pd.read_csv('../data/processed/3_seai_miss_forest_imputation.csv')
df.drop(['BerRating', 'CO2Rating'], axis = 1, inplace=True)
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(0, 'zero')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(1, 'one')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(2, 'two')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(3, 'three')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(4, 'four')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].astype('category')

df['EnergyRating'] = df['EnergyRating'].replace('A1', 'A')
df['EnergyRating'] = df['EnergyRating'].replace('A2', 'A')
df['EnergyRating'] = df['EnergyRating'].replace('A3', 'A')
df['EnergyRating'] = df['EnergyRating'].replace('B1', 'B')
df['EnergyRating'] = df['EnergyRating'].replace('B2', 'B')
df['EnergyRating'] = df['EnergyRating'].replace('B3', 'B')
df['EnergyRating'] = df['EnergyRating'].replace('C1', 'C')
df['EnergyRating'] = df['EnergyRating'].replace('C2', 'C')
df['EnergyRating'] = df['EnergyRating'].replace('C3', 'C')
df['EnergyRating'] = df['EnergyRating'].replace('D1', 'D')
df['EnergyRating'] = df['EnergyRating'].replace('D2', 'D')
df['EnergyRating'] = df['EnergyRating'].replace('E1', 'E')
df['EnergyRating'] = df['EnergyRating'].replace('E2', 'E')
df['EnergyRating'] = df['EnergyRating'].replace('F', 'F')
df['EnergyRating'] = df['EnergyRating'].replace('G', 'G')

In [3]:
new_cols = [col for col in df.columns if col != 'EnergyRating'] + ['EnergyRating']
df = df[new_cols]
del(new_cols)

In [4]:
df.EnergyRating.value_counts()

C2    124530
C3    118185
D1    114352
C1    113782
D2     98187
B3     77970
G      66815
E1     56631
A3     51193
F      46347
E2     44780
A2     43242
B2     32814
B1     15261
A1      1258
Name: EnergyRating, dtype: int64

In [5]:
df_copy = df.copy()

In [6]:
X = df.iloc[:, :-1] # Independent Variables
y = df.iloc[:, -1] # Dependent Variables

In [7]:
numerical = [
    'YearofConstruction'
    ,'GroundFloorArea(sq m)'
    ,'TotalDeliveredEnergy'
    ,'InsulationThickness'
]

X[numerical] = MinMaxScaler().fit_transform(X[numerical])

In [8]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y,  random_state=2814)

In [9]:
# Create the oversampler
smotenc = SMOTENC([2,4,5,6,7,8,9,10],random_state = 2814)
X_train, y_train = smotenc.fit_resample(X_train, y_train)

In [10]:
# https://stackoverflow.com/a/52935270/5923619
# One Hot Encodes our categorical feature and binds it to the original dataset
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [11]:
# One Hot Encode all of our categorical features
categorical = [ 
    'CountyName'
    ,'DwellingTypeDescr'
    ,'MainSpaceHeatingFuel'
    ,'MainWaterHeatingFuel'
    ,'VentilationMethod'
    ,'StructureType'
    ,'InsulationType'
    ,'NoOfSidesSheltered'
]

for feature in categorical:
    X_train = encode_and_bind(X_train, feature)

In [12]:
# One Hot Encode all of our categorical features
for feature in categorical:
    X_test = encode_and_bind(X_test, feature)

# Define SMOTE-Tomek Links
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'), smote=SMOTE(k_neighbors=6), random_state=2814)
X_train, y_train = resample.fit_resample(X_train, y_train)

In [14]:
X_test = X_test.drop('MainWaterHeatingFuel_Electricity - On-peak Night-Ra', axis=1)
X_train = X_train.drop('MainSpaceHeatingFuel_Bioethanol from renewable sour', axis=1)
X_train = X_train.drop('MainWaterHeatingFuel_None', axis=1)

In [ ]:
y_train.value_counts()

In [16]:
# Modelling
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          A1       0.37      0.62      0.46       314
          A2       0.90      0.91      0.90     10810
          A3       0.90      0.87      0.88     12798
          B1       0.53      0.70      0.60      3815
          B2       0.60      0.67      0.63      8203
          B3       0.73      0.71      0.72     19493
          C1       0.77      0.75      0.76     28446
          C2       0.76      0.75      0.75     31133
          C3       0.75      0.74      0.74     29546
          D1       0.78      0.77      0.78     28588
          D2       0.81      0.79      0.80     24547
          E1       0.74      0.77      0.75     14158
          E2       0.75      0.79      0.77     11195
           F       0.81      0.83      0.82     11587
           G       0.95      0.93      0.94     16704

    accuracy                           0.78    251337
   macro avg       0.74      0.77      0.76    251337
weighted avg       0.78   

In [17]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          A1       0.16      0.51      0.24       314
          A2       0.82      0.84      0.83     10810
          A3       0.78      0.76      0.77     12798
          B1       0.23      0.45      0.30      3815
          B2       0.23      0.40      0.29      8203
          B3       0.28      0.36      0.32     19493
          C1       0.32      0.31      0.31     28446
          C2       0.30      0.25      0.28     31133
          C3       0.28      0.23      0.25     29546
          D1       0.28      0.22      0.25     28588
          D2       0.26      0.21      0.23     24547
          E1       0.18      0.22      0.20     14158
          E2       0.17      0.23      0.20     11195
           F       0.22      0.24      0.23     11587
           G       0.59      0.51      0.55     16704

    accuracy                           0.33    251337
   macro avg       0.34      0.38      0.35    251337
weighted avg       0.34   

In [18]:
rf = RandomForestClassifier(criterion='entropy')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          A1       0.39      0.59      0.47       314
          A2       0.91      0.91      0.91     10810
          A3       0.90      0.88      0.89     12798
          B1       0.57      0.64      0.60      3815
          B2       0.61      0.60      0.60      8203
          B3       0.69      0.67      0.68     19493
          C1       0.70      0.69      0.70     28446
          C2       0.68      0.67      0.68     31133
          C3       0.65      0.65      0.65     29546
          D1       0.67      0.67      0.67     28588
          D2       0.68      0.69      0.68     24547
          E1       0.59      0.59      0.59     14158
          E2       0.58      0.58      0.58     11195
           F       0.64      0.67      0.65     11587
           G       0.86      0.86      0.86     16704

    accuracy                           0.70    251337
   macro avg       0.68      0.69      0.68    251337
weighted avg       0.70   

In [19]:
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

from sklearn.preprocessing import OneHotEncoder


categorical = [ 
    'CountyName'
    ,'DwellingTypeDescr'
    ,'MainSpaceHeatingFuel'
    ,'MainWaterHeatingFuel'
    ,'VentilationMethod'
    ,'StructureType'
    ,'InsulationType'
]

numerical = [
    'YearofConstruction'
    ,'GroundFloorArea(sq m)'
    ,'TotalDeliveredEnergy'
]

df[categorical] = df[categorical].apply(lambda series: pd.Series(
OneHotEncoder().fit_transform(series[series.notnull()]),
index=series[series.notnull()].index
))